In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 160
import matplotlib.pyplot as plt


days_list={16, 18, 21, 21.5, 23, 23.5, 24, 25, 32, 33, 35, 35.5, 37, 37.5, 38, 39, 46, 47, 50, 50.5, 52, 52.5, 53, 54}

In [2]:
counts=pd.read_csv('combined_data/counts.csv', index_col=0)
counts.index.name=None
counts.head()

,1,2,3,4,5,6,7,8,9,10,...,660,661,662,663,664,665,666,667,668,669
OTU_1,1830,199,489,1056,1474,3227,7637,4237,5597,6129,...,10032,14865,16788,23613,20729,37601,38306,21944,12971,24560
OTU_2,2929,9880,24422,17184,11193,15984,21915,13035,15798,14811,...,13360,13929,22476,12834,13951,7751,8666,5790,8536,18432
OTU_3,213,555,878,4879,5842,8939,11968,12345,7441,4962,...,11,0,21,36,26,120,21,502,1034,5944
OTU_4,9,32,205,264,203,371,1124,906,1159,988,...,28,20,49,22,19,25,23,24,20,45
OTU_5,2,7,71,84,895,1121,2338,2171,3073,3616,...,339,1234,3538,4023,4004,1284,6406,6281,3988,10769


In [3]:
taxonomy=pd.read_csv('otu_taxonomy.csv', index_col=0)
taxonomy=taxonomy.fillna('unknown')
taxonomy.head()

FileNotFoundError: [Errno 2] No such file or directory: 'otu_taxonomy.csv'

In [ ]:
taxonomy_temp=taxonomy.copy()
taxonomy_temp.insert(8, 'OTU', pd.Series(np.array(taxonomy.index), index=taxonomy.index))
index=pd.MultiIndex.from_frame(taxonomy_temp)
meta=pd.read_csv('combined_data/metadata_modified_window.csv', index_col=0)
meta.index.name = 'sample'
meta.reset_index(inplace=True)
columns=pd.MultiIndex.from_frame(meta)
c_names=['sample', 'isIncluded', 'subjectID', 'measurementID', 'perturbID', 'window', 'cohort']
print(columns)
print(c_names)

In [ ]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)
df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()
df_main.head()

In [ ]:
df_main = df_main.loc[:,df_main.columns.get_level_values(0)=='healthy']
df_main.head()

In [ ]:
index_out=df_main.index.get_level_values(5).values + ' ' + df_main.index.get_level_values(6).values + ' ' + df_main.index.get_level_values(8).values 
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)
counts_out.head()

In [ ]:
meta_out=df_main.columns.to_frame(index=None)
meta_out

In [ ]:
counts_out.to_csv('out/healthy_otu_counts.csv')
meta_out.to_csv('out/healthy_otu_meta.csv')

In [ ]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)
df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()
df_main = df_main.loc[:,df_main.columns.get_level_values(0)=='uc']
#df_main = df_main.loc[:,df_main.columns.get_level_values(2)!=10]
#df_main = df_main.loc[:,df_main.columns.get_level_values(2)!=6]

index_out=df_main.index.get_level_values(5).values + ' ' + df_main.index.get_level_values(6).values + ' ' + df_main.index.get_level_values(8).values 
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/uc_otu_counts.csv')
meta_out.to_csv('out/uc_otu_meta.csv')
meta_out

In [ ]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['phylum',	'class',	'order', 'family'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()

df_main = df_main.loc[:,df_main.columns.get_level_values(0)=='healthy']


top10=df_main.mean(1).sort_values(ascending=False)[0:10].index

df_main_rel=df_main/df_main.sum()
ax1 = df_main_rel.loc[top10].transpose().plot.bar(stacked=True)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

index_out=df_main.index.get_level_values(0).values + ' ' + df_main.index.get_level_values(1).values + ' ' + df_main.index.get_level_values(2).values + ' ' + df_main.index.get_level_values(3).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/healthy_of_counts.csv')
meta_out.to_csv('out/healthy_of_meta.csv')
meta_out

###

df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['phylum',	'class',	'order', 'family'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()

df_main = df_main.loc[:,df_main.columns.get_level_values(0)=='uc']
#df_main = df_main.loc[:,df_main.columns.get_level_values(2)!=10]
#df_main = df_main.loc[:,df_main.columns.get_level_values(2)!=6]

df_main_rel=df_main/df_main.sum()
ax2 = df_main_rel.loc[top10].transpose().plot.bar(stacked=True)
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


index_out=df_main.index.get_level_values(0).values + ' ' + df_main.index.get_level_values(1).values + ' ' + df_main.index.get_level_values(2).values + ' ' + df_main.index.get_level_values(3).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/uc_of_counts.csv')
meta_out.to_csv('out/uc_of_meta.csv')
meta_out
counts_out

In [ ]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['phylum'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()

df_main = df_main.loc[:,df_main.columns.get_level_values(0)=='healthy']


index_out=df_main.index.get_level_values(0).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/healthy_phylum_counts.csv')
meta_out.to_csv('out/healthy_phylum_meta.csv')
meta_out

###

df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['phylum'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()

df_main = df_main.loc[:,df_main.columns.get_level_values(0)=='uc']


index_out=df_main.index.get_level_values(0).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/uc_phylum_counts.csv')
meta_out.to_csv('out/uc_phylum_meta.csv')
meta_out
counts_out

In [ ]:
meta_out

In [ ]:
days_list={16, 18, 21, 21.5}
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['phylum',	'class',	'order', 'family'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()


index_out=df_main.index.get_level_values(0).values + ' ' + df_main.index.get_level_values(1).values + ' ' + df_main.index.get_level_values(2).values + ' ' + df_main.index.get_level_values(3).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/ss_of_counts.csv')
meta_out.to_csv('out/ss_of_meta.csv')
meta_out



In [ ]:
counts_out

In [ ]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['phylum'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(3).isin(days_list)].groupby(level=['cohort', 'subjectID', 'perturbID', 'window'], axis=1, dropna=False).sum()


index_out=df_main.index.get_level_values(0).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)

meta_out=df_main.columns.to_frame(index=None)

counts_out.to_csv('out/ss_phylum_counts.csv')
meta_out.to_csv('out/ss_phylum_meta.csv')
meta_out
counts_out